## Initialize notebook

In [ ]:
%load_ext autoreload
%matplotlib notebook

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from astropy.io import fits

import math
import numpy as np
import copy
import pickle
from tqdm import tqdm_notebook as tqdm
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d
from scipy.interpolate import CubicSpline
from scipy.stats import chi2 as chi2_dist

import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools
from darkhistory.history.tla import get_history

import main
import config

# Save data directory
save_data = True
direc = '/Users/viviesque/OneDrive - Massachusetts Institute of Technology/DarkHistory/output/'

## Constraints and functions

In [ ]:
# IGM temperature limits

# Redshift in first column, measurement/upper limit on T_IGM in second
# The points at redshifts 3-3.4 are excluded because they lie within
#    redshifts corresponding to HeII reionization. 
# The points at redshifts 5-5.4 are excluded in order not to make our
#    constraints overly stringent.
IGM_meas =  np.array([#[3.0,1.289e4], # Walther et al.(2018)
                      #[3.2,1.186e4],
                      #[3.4,1.404e4],
                      [3.6,1.038e4],
                      [3.8,1.205e4],
                      [4.0,0.940e4],
                      [4.2,0.890e4],
                      [4.6,0.877e4],
                      #[5.0,0.533e4],
                      #[5.4,0.599e4],
                      [5.4,1.10e4], # Gaikwad et al. (2020)
                      [5.6,1.05e4],
                      [5.8,1.20e4]])
IGM_err =   np.array([#[3.0,0.180e4],  # Walther et al.(2018)
                      #[3.2,0.130e4],
                      #[3.4,0.170e4],
                      [3.6,0.310e4],
                      [3.8,0.230e4],
                      [4.0,0.220e4],
                      [4.2,0.093e4],
                      [4.6,0.130e4],
                      #[5.0,0.120e4],
                      #[5.4,0.150e4],
                      [5.4,0.16e4], # Gaikwad et al. (2020)
                      [5.6,0.21e4],
                      [5.8,0.22e4]])

# Get number of data points
Ndata = len(IGM_meas)

# Modified chi-squared, only counting positive residuals
def chi2(Tm):
    chisquared = 0
    n = 0
    
    for i, igm in enumerate(IGM_meas[:,1]):
        if Tm[i]-igm >= 0:
            chisquared += (Tm[i]-igm)**2/(IGM_err[i,1]**2)
            n += 1
    return chisquared, n

In [ ]:
#Download the range of Planck reionization models
import csv

def get_reion(model_name):
    array = []
    with open('/Users/viviesque/OneDrive - Massachusetts Institute of Technology/DarkHistory/reion_models/'+model_name+'.csv') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            array.append([float(row[0]),float(row[1])])
    array = np.array(array)
    # turn redshift into 1+redshift
    array[:,0] = 1 + array[:,0] 
    # ignore HeII reionization
    array[array[:,1]>1+phys.chi] = 1+phys.chi 
    # interpolate function
    interp_func = interp1d(array[:,0], array[:,1], bounds_error=False, fill_value=(1+phys.chi,0))
    
    return interp_func

Planck_flex_early = get_reion('Planck_FlexKnot_early')
Planck_flex_late = get_reion('Planck_FlexKnot_late')
Planck_tanh_early = get_reion('Planck_Tanh_early')
Planck_tanh_late = get_reion('Planck_Tanh_late')

testrange = np.linspace(1,30,100)

plt.figure()
plt.plot(testrange, Planck_flex_early(testrange), color= 'r')
plt.plot(testrange, Planck_flex_late(testrange), color= 'r', linestyle='--')
plt.plot(testrange, Planck_tanh_early(testrange), color= 'darkred')
plt.plot(testrange, Planck_tanh_late(testrange), color= 'darkred', linestyle='--')
plt.axvline(6.9,linestyle='--', color='k', linewidth=1)
plt.show()

# Find difference in constraints due to (not) including HeIII

In [ ]:
# Function to get temperature at relevant redshifts for decays
def get_T_decay(mDM, tau, br, pri, reion_rs, xe_reion_func):
    """Gets the temperature at specified redshift z, decays.
    
    Parameters
    ----------
    mDM : float
        The mass of the dark matter in eV. 
    tau : float
        The decay lifetime in s. 
    br : bool
        Whether to use backreaction or not. 
    pri : {'elec', 'phot'}
        The primary particles that DM decays to. 
        
    Returns
    --------
    float
        The temperature in K at z. 
    
    """
    
    if pri == 'elec':
        pri_str = 'elec_delta'
        cf = 12
    if pri == 'phot':
        pri_str = 'phot_delta'
        cf = 4
        
    if br:
        result = main.evolve(      
            primary = pri_str,
            DM_process = 'decay',
            mDM = mDM,
            lifetime = tau,
            start_rs = 3000, end_rs=4.01,
            reion_switch = True,
            reion_rs = reion_rs,
            xe_reion_func = xe_reion_func,
            heat_switch = False,
            reion_method = None,
            photoion_rate_func=None,
            photoheat_rate_func=None,
            compute_fs_method = 'He',
            helium_TLA = True,
            backreaction = True,
            coarsen_factor = cf,
        )
        
        return interp1d(result['rs'], result['Tm']/phys.kB)(1+IGM_meas[:,0])
    
    else:
        # get_history takes a redshift vector 
        rs_vec = np.flipud(np.arange(4, 3000., 0.1))
        
        result = get_history(
            rs_vec, baseline_f = True,
            inj_particle = pri, DM_process = 'decay',
            mDM=mDM, lifetime = tau
        )
        
        return interp1d(rs_vec, result[:,0]/phys.kB)(1+IGM_meas[:,0])

In [ ]:
# Choose decay channel to electrons
pri='elec'
inj_type = 'decay'

# Make grid of masses/lifetimes
log10_m_chi_arr = np.arange(6.01, 12.78, 0.25)
log10_tau_arr = np.arange(18., 30., 0.1)

In [ ]:
# If you have constraint data from the same channel, 
#    load it here to speed up finding the new constraints
chi2_decay_br_PT_early    = pickle.load(open(direc+'constraint_data/'+pri+'_'+inj_type+'_br_Planck_Tanh_early_chi2.dat','rb'))

start_index = np.zeros_like(log10_m_chi_arr, dtype=int)
for i, mDM in enumerate(log10_m_chi_arr):
    start_index[i] = np.squeeze(np.where(chi2_decay_br_PT_early[i] != 0))[-1]
print(start_index)

At this point, save each of these files under a different name, for example "main_orig.py".
1. main.py
2. darkhistory/history/tla.py
3. darkhistory/low_energy/lowE_deposition.py
4. darkhistory/low_energy/lowE_photons.py

Then copy over the versions that keep track of ionization of HeII to HeIII, i.e. type into your terminal "cp main_HeIII.py main.py". Don't forget to change this back when you're finished running this cross-check.

In [ ]:
# Create arrays for the calculated temperatures, chi-squared, and degrees of freedom
#    at each mass/cross-section grid point
temp_HeIII = np.zeros((log10_m_chi_arr.size, log10_tau_arr.size, IGM_meas[:,0].size))
chi2_HeIII = np.zeros((log10_m_chi_arr.size, log10_tau_arr.size))
dof_HeIII = np.zeros_like(chi2_HeIII)

for i, log10mDM in enumerate(tqdm(log10_m_chi_arr)):
    print('****** log10(mDM): ', log10mDM, ' ******')
    mDM = 10**log10mDM
    
    # Variables to keep track of when chi-squared is above/below target value
    over_fit = False
    under_fit = False
    
    # Estimated starting index
    j = start_index[i]
    
    while (
        not over_fit or not under_fit
    ):
        tau = 10**log10_tau_arr[j]
        
        # Get temperatures at redshifts where there is data
        temp_HeIII[i,j,:] = get_T_decay(mDM, tau, br=True, pri=pri, reion_rs=30, xe_reion_func=Planck_tanh_early)
        # Calculate chi-squared
        chi2_HeIII[i,j], dof_HeIII[i,j] = chi2(temp_HeIII[i,j,:])
        print('lifetime: {:03.1e}'.format(tau), ' $\Chi^2$ w/ backreaction: {:03.1f} '.format(chi2_HeIII[i,j]))
        
        target = chi2_dist.ppf(0.95, Ndata/2)
        
        # If chi-squared is above target, then DM is heating the IGM too much
        #    and we need to increase the lifetime
        if chi2_HeIII[i,j] >= target:
            over_fit = True
            j += 1
        # and vice versa
        elif chi2_HeIII[i,j] == 0 or chi2_HeIII[i,j] < target:
            under_fit = True
            j -= 1
        
# Save the data that was just produced
if save_data:
    pickle.dump(temp_HeIII, open(direc+pri+'_decay_br_Planck_Tanh_early_temp_HeIII.dat','wb'))
    pickle.dump(chi2_HeIII, open(direc+pri+'_decay_br_Planck_Tanh_early_chi2_HeIII.dat','wb'))
    pickle.dump(dof_HeIII, open(direc+pri+'_decay_br_Planck_Tanh_early_dof_HeIII.dat','wb'))
print('###### Calculation Complete! ######')           

In [ ]:
### RUN THIS CELL TO LOAD DATA, IF YOU'VE ALREADY RUN THE CELLS ABOVE ###

# pri = 'elec'
# inj_type = 'decay'

# log10_m_chi_arr = np.arange(6.01, 12.78, 0.25)
# log10_tau_arr = np.arange(18., 30., 0.1)

# chi2_decay_br_PT_early = pickle.load(open(direc+'constraint_data/'+pri+'_'+inj_type+'_br_Planck_Tanh_early_chi2.dat','rb'))
# chi2_HeIII = pickle.load(open(direc+pri+'_decay_br_Planck_Tanh_early_chi2_HeIII.dat','rb'))

In [ ]:
# Function to interpolate to get constraint from IGM temperature
def log10_tau_interpolate(chi2):
    log10_taus_interp = np.zeros_like(chi2[:,0])
    
    # For each mass
    for i, elem in enumerate(chi2[:,0]):
        chi2_target = chi2_dist.ppf(0.95, Ndata/2)
        
        # Find the largest lifetime for which chi-squared is exceeds the target value
        above = np.where((chi2[i,:] > chi2_target) & (chi2[i,:]!=0.))[-1]
        below=above+1
        
        chi2s = [chi2[i,below][0], chi2[i,above][0]]
        log10_taus = [log10_tau_arr[below][0], log10_tau_arr[above][0]]

        # Interpolate to get lifetime corresponding to target chi-squared
        log10_taus_interp[i] = (np.diff(log10_taus)/np.diff(chi2s)*(chi2_target-chi2s[0]) + log10_taus[0])
    return interp1d(log10_m_chi_arr, log10_taus_interp, kind='cubic')

PT_early_interp = log10_tau_interpolate(chi2_decay_br_PT_early)
HeIII_interp    = log10_tau_interpolate(chi2_HeIII)

In [ ]:
#CMB bounds for comparison
decay_elec_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_elec_decay.csv', delimiter=',')
decay_elec_CMB = interp1d(np.transpose(decay_elec_CMB_raw)[0,:], np.transpose(decay_elec_CMB_raw)[1,:])

def xsec_bound_elec_CMB(mDM, DM_process):
    if DM_process == 'swave':
        return p_ann*(mDM*1e-9)/f_elec_CMB(np.log10(mDM), np.log10(601))[0]
    elif DM_process == 'decay':
        return np.array([decay_elec_CMB(mDM*1e-9)])[0]

log10mDM_arr_fine = np.arange(log10_m_chi_arr[0], log10_m_chi_arr[-1], 0.01)

plt.figure(figsize=[6.2,6.2])
ax = plt.gca()
ax.loglog()

xmin = 10**log10mDM_arr_fine[0]
xmax = 10**log10mDM_arr_fine[-1]
ymin = 1e20 
ymax = 1e26

plt.title(r'Decay Constraint, $\chi \to e^+ e^-$')
plt.xlabel(r'Dark Matter Mass, $m_{\chi}$ [eV]')
plt.ylabel(r'Minimum Lifetime, $\tau$ [s]')
plt.axis([xmin, xmax, ymin, ymax])

plt_CMB, = plt.plot(10**log10mDM_arr_fine, xsec_bound_elec_CMB(10**log10mDM_arr_fine, 'decay'),
                    label='CMB', color='k', linestyle='--', zorder=1)
plt_tearly, = plt.plot(10**log10mDM_arr_fine, 10**PT_early_interp(log10mDM_arr_fine), label='`conservative\' $\dot{T}^\star$',
                      color='b', linewidth='2', zorder=2)
plt_HeIII,  = plt.plot(10**log10mDM_arr_fine, 10**HeIII_interp(log10mDM_arr_fine), label='`conservative\' $\dot{T}^\star$ w/HeIII',
                     color='lightblue', linewidth='2', zorder=3)

leg=plt.legend(loc='best')

plt.show()
plt.savefig('/Users/viviesque/OneDrive - Massachusetts Institute of Technology/DarkHistory/output/IGMconstraints_elec_decay_HeIII_uncertainty.pdf')

# P-wave boost factor check

In [ ]:
# Load densities
rho_bg = np.genfromtxt(direc+"boost_factors/pwave_bg.txt",delimiter=",")
rho_einasto = np.genfromtxt(direc+"boost_factors/pwave_einasto.txt",delimiter=",")
rho_NFW = np.genfromtxt(direc+"boost_factors/pwave_NFW.txt",delimiter=",")

# Interpolate
rho_bg = interp1d(rho_bg[:,0],rho_bg[:,1], bounds_error=False, fill_value=1)
rho_einasto = interp1d(rho_einasto[:,0],rho_einasto[:,1], bounds_error=False, fill_value=1)
rho_NFW = interp1d(rho_NFW[:,0],rho_NFW[:,1], bounds_error=False, fill_value=1)

# Plot to compare to Fig. 3 of Liu, Slatyer, and Zavala (2016)
testrange = np.logspace(-0.97,1.85,100)

plt.figure()
plt.loglog()
plt.plot(testrange, rho_bg(testrange), color= 'k')
plt.plot(testrange, rho_NFW(testrange), color= 'b')
plt.plot(testrange, rho_einasto(testrange), color= 'r')
plt.show()

In [ ]:
# Calculate boost factors
def boost_einasto(rs):
    return (rho_einasto(rs)/rho_bg(rs))**2

def boost_NFW(rs):
    return (rho_NFW(rs)/rho_bg(rs))**2

# Plot
testrange = np.logspace(-0.97,2,100)

plt.figure()
plt.loglog()
plt.plot(testrange, boost_NFW(testrange), color= 'b')
plt.plot(testrange, boost_einasto(testrange), color= 'r')
plt.show()

In [ ]:
# Choose p-wave annihilation channel to electrons
pri = 'elec'
inj_type = 'pwave'

# Make grid of masses/cross-sections
log10_m_chi_arr = np.arange(6.01, 12.78, 0.25)    
log10_sigmav_over_m_arr = np.arange(-28.01, -15., 0.1)

In [ ]:
# If you have constraint data from the same channel, 
#    load it here to speed up finding the new constraints
chi2_decay_br_PF_early    = pickle.load(open(direc+'constraint_data/'+pri+'_'+inj_type+'_br_Planck_Flex_early_chi2.dat','rb'))

start_index = np.zeros_like(log10_m_chi_arr, dtype=int)
for i, mDM in enumerate(log10_m_chi_arr):
    arr = np.squeeze(np.where(chi2_decay_br_PF_early[i] != 0))
    if (arr.size > 1):
        start_index[i] = np.squeeze(np.where(chi2_decay_br_PF_early[i] != 0))[0]
    else:
        start_index[i] = np.squeeze(np.where(chi2_decay_br_PF_early[i] != 0))
print(start_index)

In [ ]:
# Function that calculates temperature histories and chi-squared, given a boost factor
def constraint_from_boost(struct_boost):
    # Create arrays for the calculated temperatures, chi-squared, and degrees of freedom
    #    at each mass/cross-section grid point
    temps = np.zeros((log10_m_chi_arr.size, log10_sigmav_over_m_arr.size, IGM_meas[:,0].size))
    chi2s = np.zeros((log10_m_chi_arr.size, log10_sigmav_over_m_arr.size))
    dofs = np.zeros_like(chi2s)
    
    for i, log10mDM in enumerate(tqdm(log10_m_chi_arr)): 
        print('****** log10(mDM): ', log10mDM, ' ******')
        mDM = 10**log10mDM

        # Variables to keep track of when chi-squared is above/below target value
        over_fit = False
        under_fit = False

        # Estimated starting index
        j = start_index[i]

        while (
            not over_fit or not under_fit
        ):
            sigmav = 10**log10_sigmav_over_m_arr[j] * mDM/1e9

            result = main.evolve(
                primary='elec_delta',
                DM_process='pwave', mDM=mDM, sigmav=sigmav,
                reion_switch=True, reion_rs=35, helium_TLA=True,
                xe_reion_func = Planck_flex_early,
                start_rs = 3000, end_rs=4.01, 
                heat_switch = False, equibheat_switch = False,
                coarsen_factor=12, backreaction=True, 
                compute_fs_method='He', mxstep=1000, rtol=1e-4,
                use_tqdm=True, cross_check = False,
                struct_boost=struct_boost
            )          

            # Extract temperatures at the relevant redshifts
            temps[i,j,:] = interp1d(result['rs'], result['Tm']/phys.kB)(1+IGM_meas[:,0])
            # Calculate chi-squared
            chi2s[i,j], dofs[i,j] = chi2(temps[i,j,:])
            print('lifetime: {:03.1e}'.format(sigmav), ' $\Chi^2$ : {:03.1f} '.format(chi2s[i,j]))

            # Check if chi-squared is above/below targed value
            target = chi2_dist.ppf(0.95, Ndata/2)

            # If chi-squared is too large, then temperature is too high
            #    and we need to lower the cross-section so DM adds less heat
            if chi2s[i,j] >= target:
                over_fit = True
                j -= 1
            # and vice versa
            elif chi2s[i,j] == 0 or chi2s[i,j] < target:
                under_fit = True
                j += 1
                
    print('###### Calculation Complete! ######')   
    return temps, chi2s, dofs      

In [ ]:
# Calculate chi-squared using Einasto boost factor
temps_einasto, chi2s_einasto, dofs_einasto = constraint_from_boost(boost_einasto)

# Save the data that was just produced
if save_data:
    pickle.dump(chi2s_einasto, open(direc+pri+'_pwave_einasto.dat','wb'))

In [ ]:
# Calculate chi-squared using NFW boost factor
temps_NFW, chi2s_NFW, dofs_NFW = constraint_from_boost(boost_NFW)

# Save the data that was just produced
if save_data:
    pickle.dump(chi2s_NFW, open(direc+pri+'_pwave_NFW.dat','wb'))

In [ ]:
### RUN THIS CELL TO LOAD DATA, IF YOU'VE ALREADY RUN THE CELLS ABOVE ###

# pri = 'elec'
# inj_type = 'pwave'

# log10_m_chi_arr = np.arange(6.01, 12.78, 0.25)    
# log10_sigmav_over_m_arr = np.arange(-28.01, -15., 0.1)

# chi2s_einasto = pickle.load(open(direc+pri+'_pwave_einasto.dat','rb'))
# chi2s_NFW = pickle.load(open(direc+pri+'_pwave_NFW.dat','rb'))

In [ ]:
# Interpolate chi-squared values for each mass to find cross-section
#    that gives the target chi-squared value
def log10_sigmav_interpolate(chi2):
    log10_sigs_interp = np.zeros_like(chi2[:,0])
    for i, elem in enumerate(chi2[:,0]):
        chi2_target = chi2_dist.ppf(0.95, Ndata/2)
        
        # Find the smallest cross-section where the calculated chi-squared exceeds the target
        above = np.where((chi2[i,:] > chi2_target) & (chi2[i,:]!=0.))[0]
        
        # Check that there is such a cross-section
        # Then get the largest cross-section with chi-squared below target
        if len(above) != 0:
            above = above[0]
            below = above-1
        else:
            below = np.where((chi2[i,:] < chi2_target) & (chi2[i,:]!=0.))[0][0]
            above = below-1

        # If both of these are non-zero, then interpolate to get target cross-section
        # Otherwise, leave target cross-section as 0
        if (chi2[i,above] != 0.) and (chi2[i,below] != 0.):
            chi2s = [chi2[i,below], chi2[i,above]]
            log10_sigs = [log10_sigmav_over_m_arr[below], log10_sigmav_over_m_arr[above]]

            log10_sigs_interp[i] = (np.diff(log10_sigs)/np.diff(chi2s)*(chi2_target-chi2s[0]) + log10_sigs[0])
    return log10_sigs_interp

# Calculate cross-sections corresponding to target chi-squared value
sigs_einasto = log10_sigmav_interpolate(chi2s_einasto)
sigs_NFW = log10_sigmav_interpolate(chi2s_NFW)

# Make interpolating functions
interp_einasto = interp1d(log10_m_chi_arr, sigs_einasto)
interp_NFW = interp1d(log10_m_chi_arr, sigs_NFW)

In [ ]:
# Run this if you want to save the cross-sections you calculated above
pickle.dump(sigs_einasto, open(direc+pri+'_pwave_sigs_einasto.dat','wb'))
pickle.dump(sigs_NFW, open(direc+pri+'_pwave_sigs_NFW.dat','wb'))

In [ ]:
# Plot
log10mDM_arr_fine = np.arange(log10_m_chi_arr[0], log10_m_chi_arr[-1], 0.01)

plt.figure(figsize=[6.2,6.2])
ax = plt.gca()
ax.loglog()

xmin = 10**log10mDM_arr_fine[0]
xmax = 10**log10mDM_arr_fine[-1]

plt.title(r'p-wave constraint, $\chi\chi \to e^+ e^-$')
ymin = 1e-28 
ymax = 1e-15

plt.xlabel(r'Dark matter mass, $m_{\chi}$ [eV]')
plt.ylabel(r'Maximum Cross-Section $(\sigma v)_{ref}$ [cm$^3$ s$^{-1}$]')
plt.axis([xmin, xmax, ymin, ymax])

plt_einasto, = plt.plot(10**log10mDM_arr_fine, (10**interp_einasto(log10mDM_arr_fine))*(10**log10mDM_arr_fine)/1e9, 
                        label='Einasto boost factor', color='r')
plt_NFW, = plt.plot(10**log10mDM_arr_fine, (10**interp_NFW(log10mDM_arr_fine))*(10**log10mDM_arr_fine)/1e9, 
                        label='NFW boost factor', color='b')

leg=plt.legend(loc='best')

plt.show()

In [ ]:
# Print the maximum difference between the constraints made
#    using the different boost factors
ein_array = interp_einasto(log10mDM_arr_fine)
nfw_array = interp_NFW(log10mDM_arr_fine)
print(np.max((nfw_array-ein_array)/nfw_array))

# Optical Depth Checks

In [ ]:
from scipy.integrate import quad

# Functions to calculate optical depth
def integrand(rs, xe_func):
    return phys.nH * phys.c * phys.thomson_xsec * xe_func(rs) * rs**2/phys.hubble(rs)

def get_opdepth_int(rs_min, rs_max, xe_func):
    return quad(integrand, rs_min, rs_max, args=(xe_func,))

In [ ]:
# Function to get ionization history only due to DM at each grid point
def get_history_decay(mDM, tau, br, pri, reion_rs, xe_reion_func):
    """Gets the temperature at specified redshift z, decays.
    
    Parameters
    ----------
    mDM : float
        The mass of the dark matter in eV. 
    tau : float
        The decay lifetime in s. 
    br : bool
        Whether to use backreaction or not. 
    pri : {'elec', 'phot'}
        The primary particles that DM decays to. 
        
    Returns
    --------
    float
        The temperature in K at z. 
    
    """
    
    if pri == 'elec':
        pri_str = 'elec_delta'
        cf = 12
    if pri == 'phot':
        pri_str = 'phot_delta'
        cf = 4
        
    result = main.evolve(      
        primary = pri_str,
        DM_process = 'decay',
        mDM = mDM,
        lifetime = tau,
        start_rs = 3000, end_rs=4.01,
        heat_switch = False,
        reion_method = None,
        photoion_rate_func=None,
        photoheat_rate_func=None,
        compute_fs_method = 'He',
        helium_TLA = True,
        backreaction = True,
        coarsen_factor = cf,
    )

    return result

## Get constraint from Planck 2018 optical depth measurement

The 68% constraint on the optical depth from Planck 2018 using tanh functions is $\tau = 0.0519 +0.0030$. Remember that we are using the integration variable $1+z$. We will subtract off the contributions from $z \in [3,6]$ and $z \in [0,3]$ by assuming instantaneous full reionization of HI and HeI, then HeII (this is very close to using the late Tanh reionization history).

In [ ]:
def reion_first(rs):
    return 1 + phys.chi
def reion_second(rs):
    return 1 + 2*phys.chi

opdepth_reion1 = get_opdepth_int(4, 7, reion_first)
opdepth_reion2 = get_opdepth_int(1, 4, reion_second)
print(f'Optical depth from z = (3,6): {opdepth_reion1[0]:0.4f}')
print(f'Optical depth from z = (0,3): {opdepth_reion2[0]:0.4f}')

opdepth_remain = 0.0519 - opdepth_reion1[0] - opdepth_reion2[0]
opdepth_err = 0.003
print(f'Remaining optical depth: {opdepth_remain:0.4f}')

opdepth_target = opdepth_remain + opdepth_err

The constraints on dark matter will come from where $\tau > 0.0135+0.0030 = 0.0165$.

In [ ]:
# Choose decay channel to electrons
pri = 'elec'
inj_type = 'decay'

# Make grid of masses/lifetimes
log10_m_chi_arr = np.arange(6.01, 12.78, 0.25)
log10_tau_arr = np.arange(18., 30., 0.1)

In [ ]:
# If you have constraint data from the same channel, 
#    load it here to speed up finding the new constraints
chi2_decay_br_PT_early    = pickle.load(open(direc+'constraint_data/'+pri+'_'+inj_type+'_br_Planck_Tanh_early_chi2.dat','rb'))

start_index = np.zeros_like(log10_m_chi_arr, dtype=int)
for i, mDM in enumerate(log10_m_chi_arr):
    start_index[i] = np.squeeze(np.where(chi2_decay_br_PT_early[i] != 0))[-1]
print(start_index)

In [ ]:
# Get constraints from optical depth measurement
opdepth_grid = np.zeros((log10_m_chi_arr.size, log10_tau_arr.size))

for i, log10mDM in enumerate(tqdm(log10_m_chi_arr)):
    print('****** log10(mDM): ', log10mDM, ' ******')
    mDM = 10**log10mDM
    
    # Variables to keep track of when we go above/below the target optical depth
    above = False
    below = False
    
    # Estimated starting index
    if i == 0 :
        j = start_index[i] + 8
    
    while (
        not above or not below
    ):
        tau = 10**log10_tau_arr[j]
        
        # Calculate ionization history
        result = get_history_decay(mDM, tau, br=True, pri=pri, reion_rs=30, xe_reion_func=Planck_tanh_late)
        xe_result = interp1d(result['rs'], result['x'][:,0] + result['x'][:,1], bounds_error=False, fill_value=1+phys.chi)
        
        # Calculate optical depth from 1+z = [7,50]
        opdepth_grid[i,j] = get_opdepth_int(7, 50, xe_result)[0]
        
        print('lifetime: {:03.1e}'.format(tau), ' optical depth: {:03.4f} '.format(opdepth_grid[i,j])) 
        
        # If optical depth is too high, then DM is causing too much ionization
        #    and we need to increase the lifetime
        if opdepth_grid[i,j] >= opdepth_target:
            above = True
            j += 1
        # and vice versa
        elif opdepth_grid[i,j] < opdepth_target:
            below = True
            j -= 1
        
# Save the data that was just produced
if save_data:
    pickle.dump(opdepth_grid, open(direc+pri+'_decay_br_Planck_Tanh_early_opdepth_true.dat','wb'))
print('###### Calculation Complete! ######')           

## Recreate 2016 optical depth constraint

The 68% constraint on the optical depth from Planck 2015 using tanh functions is $\tau = 0.058 + 0.012$. Remember that we are using the integration variable $1+z$. We will subtract off the contributions from $z \in [3,6]$ and $z \in [0,3]$ by assuming instantaneous full reionization of HI and HeI, then HeII, as well as early contributions by subtracting off a history without dark matter.

In [ ]:
def reion_first(rs):
    return 1 + phys.chi
def reion_second(rs):
    return 1 + 2*phys.chi

opdepth_reion1 = get_opdepth_int(4, 7, reion_first)
opdepth_reion2 = get_opdepth_int(1, 4, reion_second)
print(f'Optical depth from z = (3,6): {opdepth_reion1[0]:0.3f}')
print(f'Optical depth from z = (0,3): {opdepth_reion2[0]:0.3f}')
print(f'Optical depth from z = (0,6): {opdepth_reion1[0]+opdepth_reion2[0]:0.3f}')

opdepth_remain = 0.058 - opdepth_reion1[0] - opdepth_reion2[0]
opdepth_err = 0.012
print(f'Remaining optical depth: {opdepth_remain:0.4f}')

opdepth_target2016 = opdepth_remain + 2*opdepth_err
print(rf'The constraints on dark matter will come from where tau > {opdepth_remain:0.3f} + 2*{opdepth_err:0.3f} = {opdepth_target2016:0.3f}.')

In [ ]:
# Get constraints from 2016 optical depth measurement
opdepth_grid2016 = np.zeros((log10_m_chi_arr.size, log10_tau_arr.size))
rs_vec = np.arange(4, 3000., 0.1)

for i, log10mDM in enumerate(tqdm(log10_m_chi_arr)):
    print('****** log10(mDM): ', log10mDM, ' ******')
    mDM = 10**log10mDM   
    above = False
    below = False
    
    if i == 0 :
        j = start_index[i] + 8
    
    while (
        not above or not below
    ):
        tau = 10**log10_tau_arr[j]
        result = get_history_decay(mDM, tau, br=False, pri=pri, reion_rs=30, xe_reion_func=Planck_tanh_late)
        
        # Calculate ionization history from a universe without DM
        subtract = get_history(np.flipud(rs_vec))
        
        # Subtract off this contribution to find ionization only due to DM
        xe_result = interp1d(np.flipud(rs_vec), result[:,1] + result[:,2] - subtract[:,1] - subtract[:,2], bounds_error=False, fill_value=1+phys.chi)
        # Calculate corresponding optical depth
        opdepth_grid2016[i,j] = get_opdepth_int(7, 1700, xe_result)[0]
        
        print('lifetime: {:03.1e}'.format(tau), ' optical depth: {:03.4f} '.format(opdepth_grid2016[i,j])) 
        
        if opdepth_grid2016[i,j] >= opdepth_target2016:
            above = True
            j += 1
        elif opdepth_grid2016[i,j] < opdepth_target2016:
            below = True
            j -= 1
        
# Save the data that was just produced
if save_data:
    pickle.dump(opdepth_grid2016, open(direc+pri+'_decay_br_Planck_Tanh_early_opdepth_true2016.dat','wb'))
print('###### Calculation Complete! ######')           

In [ ]:
### RUN THIS CELL TO LOAD DATA, IF YOU'VE ALREADY RUN THE CELLS ABOVE ###

# pri = 'elec'
# inj_type = 'decay'

# log10_m_chi_arr = np.arange(6.01, 12.78, 0.25)
# log10_tau_arr = np.arange(18., 30., 0.1)

# chi2_decay_br_PT_early    = pickle.load(open(direc+'constraint_data/'+pri+'_'+inj_type+'_br_Planck_Tanh_early_chi2.dat','rb'))
# opdepth_grid = pickle.load(open(direc+pri+'_decay_br_Planck_Tanh_early_opdepth_true.dat','rb'))
# opdepth_grid2016    = pickle.load(open(direc+pri+'_'+inj_type+'_br_Planck_Tanh_early_opdepth_true2016.dat','rb'))

# opdepth_target = 0.0165
# opdepth_target2016 = 0.0436

In [ ]:
# Function to interpolate to get constraint from optical depth
def log10_tau_interpolate_opdepth(opdepth, target):
    log10_taus_interp = np.zeros_like(opdepth[:,0])
    
    # For each mass
    for i, elem in enumerate(opdepth[:,0]):
        # Find the index of the largest lifetime where the optical depth is larger than the target value
        above = np.where(opdepth[i,:] > target)[-1]
        below=above+1
        
        opdepths = [opdepth[i,below][0], opdepth[i,above][0]]
        log10_taus = [log10_tau_arr[below][0], log10_tau_arr[above][0]]

        # Interpolate to get lifetime corresponding to target optical depth
        log10_taus_interp[i] = (np.diff(log10_taus)/np.diff(opdepths)*(target-opdepths[0]) + log10_taus[0])
    return interp1d(log10_m_chi_arr, log10_taus_interp, kind='cubic')

opdepth_interp = log10_tau_interpolate_opdepth(opdepth_grid, opdepth_target)
opdepth2016_interp = log10_tau_interpolate_opdepth(opdepth_grid2016, opdepth_target2016)

# Function to interpolate to get constraint from IGM temperature
def log10_tau_interpolate(chi2, dof):
    log10_taus_interp = np.zeros_like(chi2[:,0])
    for i, elem in enumerate(chi2[:,0]):
        chi2_target = chi2_dist.ppf(0.95, Ndata/2)
        
        # Find the largest lifetime for which chi-squared is exceeds the target value
        above = np.where((chi2[i,:] > chi2_target) & (chi2[i,:]!=0.))[-1]
        below=above+1
        
        chi2s = [chi2[i,below][0], chi2[i,above][0]]
        log10_taus = [log10_tau_arr[below][0], log10_tau_arr[above][0]]

        # Interpolate to get lifetime corresponding to target chi-squared
        log10_taus_interp[i] = (np.diff(log10_taus)/np.diff(chi2s)*(chi2_target-chi2s[0]) + log10_taus[0])
    return interp1d(log10_m_chi_arr, log10_taus_interp, kind='cubic')

PT_early_interp = log10_tau_interpolate(chi2_decay_br_PT_early, dof_decay_br_PT_early)

In [ ]:
#CMB bounds for comparison
decay_elec_CMB_raw = np.loadtxt(config.data_path+'/CMB_limits_elec_decay.csv', delimiter=',')
decay_elec_CMB = interp1d(np.transpose(decay_elec_CMB_raw)[0,:], np.transpose(decay_elec_CMB_raw)[1,:])

def xsec_bound_elec_CMB(mDM, DM_process):
    if DM_process == 'swave':
        return p_ann*(mDM*1e-9)/f_elec_CMB(np.log10(mDM), np.log10(601))[0]
    elif DM_process == 'decay':
        return np.array([decay_elec_CMB(mDM*1e-9)])[0]

log10mDM_arr_fine = np.arange(log10_m_chi_arr[0], log10_m_chi_arr[-1], 0.01)

plt.figure(figsize=[6.2,6.2])
ax = plt.gca()
ax.loglog()

xmin = 10**log10mDM_arr_fine[12]
xmax = 10**log10mDM_arr_fine[-1]
# xmin = 10**-3
# xmax = 10**3

plt.title(r'Decay Constraint, $\chi \to e^+ e^-$')
ymin = 3e21 
ymax = 5e25
# ymin = 1e18 
# ymax = 1e29

plt.xlabel(r'Dark Matter Mass, $m_{\chi}$ [eV]')
plt.ylabel(r'Minimum Lifetime, $\tau$ [s]')
plt.axis([xmin, xmax, ymin, ymax])

plt_CMB,     = plt.plot(10**log10mDM_arr_fine, xsec_bound_elec_CMB(10**log10mDM_arr_fine, 'decay'),
                      label='CMB', linestyle='--', color='k', zorder=1)
plt_tearly,  = plt.plot(10**log10mDM_arr_fine[12:], 10**PT_early_interp(log10mDM_arr_fine[12:]), label='`conservative\' $\dot{T}^\star$',
                      color='b', linewidth='2',zorder=4)
plt_opdepth, = plt.plot(10**log10mDM_arr_fine, 10**opdepth_interp(log10mDM_arr_fine), label='Optical depth',
                     color='r', linewidth='2',zorder=3)
plt_opdepth2016, = plt.plot(10**log10mDM_arr_fine, 10**opdepth2016_interp(log10mDM_arr_fine), label='Optical depth (2016)',
                      color='pink', linewidth='2', linestyle=':',zorder=2)

leg=plt.legend(loc='best')

plt.show()